In [ ]:
from typing import TypedDict, Annotated
from langgraph.graph import add_messages, StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode

load_dotenv()

In [ ]:
class ChildState(TypedDict):
    messages: Annotated[list, add_messages]

search_tool = TavilySearchResults(max_results=2)
tools = [search_tool]
llm = ChatOpenAI(model='llama-3.1')
llm_with_tools = llm.bind_tools(tools=tools)

In [ ]:
def agent(state:ChildState):
    return {
        'messages': [llm_with_tools.invoke(state['messages'])]
    }

def tools_router(state:ChildState):
    last_message = state['messages'][-1]
    if(hasattr(last_message, 'tool_calls') and len(last_message.tool_calls) > 0):
        return 'tool_node'
    else:
        return END

tool_node = ToolNode(tools=tools)
subgraph = StateGraph(ChildState)
subgraph.add_node('agent', agent)
subgraph.add_node('tool_node', tool_node)
subgraph.set_entry_point('agent')
subgraph.add_conditional_edges('agent', tools_router)
subgraph.add_edge('tool_node', 'agent')
search_app = subgraph.compile()

search_app.invoke({'messages': [HumanMessage(content='How is the weather in Bangalore')]})


In [1]:
from typing import TypedDict, Annotated, Dict
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage

class QueryState(TypedDict):
    query:str
    response:str

In [ ]:
def search_agent(state:QueryState) -> Dict:
    subgraph_input = {
        'messages': [HumanMessage(content=state['query'])]
    }
    subgraph_result = search_app.invoke(subgraph_input)
    assistant_message = subgraph_result['messages'][-1]
    return {'response': assistant_message.content}

parent_graph = StateGraph(QueryState)
parent_graph.add_node('search_agent', search_agent)
parent_graph.add_edge(START, 'search_agent')
parent_graph.add_edge('search_agent', END)

parent_app = parent_graph.compile()
result = parent_app.invoke({'query': 'How is the weather in Bangalore', 'response':''})
print(result)